In [ ]:
# Завдання 1 (Переклад тексту)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Завантаження
model_name = "Helsinki-NLP/opus-mt-en-uk"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Вхідні дані
text = "A cinematic photograph of a cyberpunk raccoon hacker sitting in front of multiple screens in a dark, neon-lit room, raining outside, highly detailed, 8k"

# Токенізація
inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

# Генерація
with torch.no_grad():
    # model.generate створює послідовність токенів перекладу
    translated_tokens = model.generate(**inputs)

# Декодування
translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

print(f"Original: {text}")
print(f"Translated: {translated_text[0]}")

In [ ]:
pip install --upgrade diffusers transformers scipy ftfy accelerate

In [ ]:
# Завдання 2 і 3с (Генерація картинки)

import torch
from diffusers import StableDiffusionPipeline
from PIL import Image

# Налаштування
# Використовуємо GPU. Якщо немає CUDA, код працюватиме дуже повільно на CPU.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_id = "runwayml/stable-diffusion-v1-5"
# model_id = "stabilityai/stable-diffusion-2-1" # Можна спробувати новішу версію

# Завантаження пайплайну
# torch_dtype=torch.float16 значно зменшує використання пам'яті GPU без втрати якості
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
pipe = pipe.to(device)

# Генерація
# Ваш запит
prompt = "A cinematic photograph of a cyberpunk raccoon hacker sitting in front of multiple screens in a dark, neon-lit room, raining outside, highly detailed, 8k"

# Негативний запит
negative_prompt = "blurry, bad art, low quality, distorted face, ugly"

# Параметри генерації:
steps = 30

guidance = 7.5

print("Генерація зображення... Зачекайте.")
image = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=steps,
    guidance_scale=guidance,
    height=512,
    width=512
).images[0]

output_filename = "cyberpunk_raccoon.png"
image.save(output_filename)
print(f"Зображення збережено у файл: {output_filename}")
image.show()

In [ ]:
# Завдання 3а (Оцінка якісті класифікації)

import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

# 1. Завантаження даних
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

# Беремо 50 рядків для тесту
test_df = df[:50].copy()

print(f"Завантажено {len(test_df)} рядків для тесту.")
print("Приклад даних:", test_df['label'].unique())

# МОДЕЛЬ 1: Zero-Shot
print("\n1. Запуск Zero-Shot Model (facebook/bart-large-mnli)...")
classifier_zero = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["spam", "ham"]

# Створюємо список для результатів
preds_zero = []
for text in tqdm(test_df['message']):
    res = classifier_zero(text, candidate_labels)
    preds_zero.append(res['labels'][0])

test_df['pred_zero'] = preds_zero

# МОДЕЛЬ 2: Specialized
print("\n2. Запуск Specialized Model (mariagrandury/roberta-base-finetuned-sms-spam-detection)...")
classifier_spam = pipeline("text-classification", model="mariagrandury/roberta-base-finetuned-sms-spam-detection")

preds_spec = []
for text in tqdm(test_df['message']):
    res = classifier_spam(text, truncation=True, max_length=512)[0]
    raw_label = res['label']

    if raw_label == 'LABEL_1':
        preds_spec.append('spam')
    elif raw_label == 'LABEL_0':
        preds_spec.append('ham')
    else:
        preds_spec.append(raw_label.lower())

test_df['pred_spec'] = preds_spec


print("\n" + "="*50)
print("ПОРІВНЯННЯ РЕЗУЛЬТАТІВ")
print("="*50)

print("Приклад прогнозів:")
print(test_df[['label', 'pred_zero', 'pred_spec']].head())

print("\n Метрики Моделі 1 (Zero-Shot) ")
print(classification_report(test_df['label'], test_df['pred_zero'], zero_division=0))
acc1 = accuracy_score(test_df['label'], test_df['pred_zero'])
print(f"Accuracy: {acc1:.4f}")

print("\n Метрики Моделі 2 (Specialized) ")
print(classification_report(test_df['label'], test_df['pred_spec'], zero_division=0))
acc2 = accuracy_score(test_df['label'], test_df['pred_spec'])
print(f"Accuracy: {acc2:.4f}")

print("\n" + "="*50)
if acc2 > acc1:
    print(f"ВИСНОВОК: Спеціалізована модель перемогла (Acc: {acc2:.2f} vs {acc1:.2f})")
else:
    print(f"ВИСНОВОК: Zero-Shot модель впоралася краще або так само (Acc: {acc1:.2f} vs {acc2:.2f})")